In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [2]:
historique = pd.read_csv("csv/HistoricalEsportData.csv",sep=',',encoding="ISO-8859-1")

In [3]:
historique

,Date,Game,Earnings,Players,Tournaments
0,1/1/1998,QuakeWorld,15000.0,8,1
1,5/1/1998,Quake II,14600.0,4,1
2,7/1/1998,Quake II,15000.0,5,1
3,7/1/1998,QuakeWorld,6500.0,4,1
4,11/1/1998,Quake II,36600.0,13,2
...,...,...,...,...,...
6210,9/1/2020,Tetr.io,200.0,4,1
6211,9/1/2020,Beat Aim,160.0,7,2
6212,9/1/2020,Tetris,125.0,2,1
6213,9/1/2020,Rivals of Aether,50.0,4,1


In [4]:
historique.isnull().sum()

Date           0
Game           0
Earnings       0
Players        0
Tournaments    0
dtype: int64

In [5]:
historique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6215 entries, 0 to 6214
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         6215 non-null   object 
 1   Game         6215 non-null   object 
 2   Earnings     6215 non-null   float64
 3   Players      6215 non-null   int64  
 4   Tournaments  6215 non-null   int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 194.3+ KB


In [6]:
historique.describe()

,Earnings,Players,Tournaments
count,6.215000e+03,6215.000000,6215.000000
mean,1.407601e+05,50.310217,6.524859
std,9.577379e+05,115.961782,12.411488
min,0.000000e+00,0.000000,1.000000
25%,1.000000e+03,5.000000,1.000000
50%,8.776800e+03,15.000000,2.000000
75%,5.216008e+04,41.000000,6.000000
max,3.671356e+07,1699.000000,172.000000


On récuppere toutes les informations lié aux jeux de la série Smash bros

On ne garde que l'année pour simplifier le traitement des données

On change les titres de jeux pour pouvoir faciliter le traitement de données

Liste des Jeux:
-   Super Smash Bros. = 0
-   Super Smash Bros. Brawl = 1
-   Super Smash Bros. for Wii U = 2
-   Super Smash Bros. Melee = 3
-   Super Smash Bros. Ultimate = 4

In [7]:
smash = historique[historique.Game == 'Super Smash Bros.']
smash.append(historique[historique.Game == 'Super Smash Bros. Brawl'])
smash.append(historique[historique.Game == 'Super Smash Bros. for Wii U'])
smash.append(historique[historique.Game == 'Super Smash Bros. Melee'])
smash.append(historique[historique.Game == 'Super Smash Bros. Ultimate'])
smash.Date = smash.Date.str[-4:]
smash.rename(columns={'Date': 'Year'}, inplace=True)
lb = LabelEncoder()
smash.Game = lb.fit_transform(smash.Game)

In [8]:
smash.head()

,Year,Game,Earnings,Players,Tournaments
628,2007,0,415.0,6,2
641,2007,0,525.0,3,1
732,2007,0,0.0,3,1
914,2009,0,100.0,7,2
1105,2010,0,40.0,3,2


In [9]:
smash.describe()

,Game,Earnings,Players,Tournaments
count,60.0,60.000000,60.000000,60.000000
mean,0.0,1134.713500,9.633333,2.016667
std,0.0,1779.796342,8.406952,1.455459
min,0.0,0.000000,1.000000,1.000000
25%,0.0,153.750000,4.000000,1.000000
50%,0.0,418.115000,7.500000,2.000000
75%,0.0,1000.000000,11.250000,2.000000
max,0.0,8860.680000,44.000000,9.000000


In [10]:
smashX = smash[['Year','Game','Players','Tournaments']]
smashY = smash[['Earnings']]

In [11]:
smashX.head()

,Year,Game,Players,Tournaments
628,2007,0,6,2
641,2007,0,3,1
732,2007,0,3,1
914,2009,0,7,2
1105,2010,0,3,2


In [12]:
smashY.head()

,Earnings
628,415.0
641,525.0
732,0.0
914,100.0
1105,40.0


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
smashX_train, smashX_test, smashY_train, smashY_test = train_test_split(smashX,smashY,test_size=0.15)
len(smashX_train)

51

In [15]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()

In [16]:
plt.scatter(smashX, smashY,color='g')
plt.plot(smashX, clf.predict(smashX),color='k')
plt.show

TypeError: unhashable type: 'numpy.ndarray'

In [747]:
clf.fit(smashX_train,smashY_train)

LinearRegression()

In [748]:
smash.Game = lb.inverse_transform(smash.Game)

In [749]:
smash_pred = clf.predict(smashX_test.head())
smash_pred

array([[ 1.51376335e+00],
       [ 2.50881943e+03],
       [ 2.90180943e+03],
       [ 4.16164264e+03],
       [-3.87199092e+02]])

In [750]:
smashY_test.head()

,Earnings
1217,110.0
3372,610.0
5439,4230.0
4658,4940.0
5888,190.0


In [751]:
clf.score(smashX_test,smashY_test)

0.7313017321779788

In [752]:
clf.coef_

array([[-3.88712855e+01,  2.84217094e-14,  1.84470482e+02,
         1.14139038e+02]])

In [755]:
smashX_test.shape

(9, 4)

In [767]:
plt.scatter(smashX_test, smashX_test,  color='black')
plt.plot(smashY_test, smashY_test, color='blue', linewidth=3)
plt.xticks(())
plt.yticks(())

plt.show()

TypeError: unhashable type: 'numpy.ndarray'